# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

话术分享 langchain/sales_chatbot/Sales Training_ Effective Insurance Communication.json


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [8]:
pip install -U langchain-openai

   ---------------------------------------- 0.0/46.7 kB ? eta -:--:--
   ----------------------------------- ---- 41.0/46.7 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 46.7/46.7 kB 588.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/337.0 kB ? eta -:--:--
   --- ------------------------------------ 30.7/337.0 kB ? eta -:--:--
   ------- -------------------------------- 61.4/337.0 kB 1.7 MB/s eta 0:00:01
   ------- -------------------------------- 61.4/337.0 kB 1.7 MB/s eta 0:00:01
   ----------------------- ---------------- 194.6/337.0 kB 1.2 MB/s eta 0:00:01
   ----------------------- ---------------- 194.6/337.0 kB 1.2 MB/s eta 0:00:01
   ------------------------- ------------ 225.3/337.0 kB 919.0 kB/s eta 0:00:01
   ------------------------- ------------ 225.3/337.0 kB 919.0 kB/s eta 0:00:01
   -------------------------------------- 337.0/337.0 kB 949.9 kB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: opena

In [1]:
with open("real_insurance_sales_data.txt", encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 350,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

In [5]:
docs[0]

Document(page_content='\\d+\\. 问：请简单介绍一下您自己？。\n   答：您好，我是“安逸养老”保险的销售顾问小陈，很高兴能够和您交流。我在保险行业有多年的工作经验，专注于为客户提供个性化的保险解决方案。\n\n\\d+\\. 问：介绍一下我们公司。\n   答：我们公司“安逸养老”专注于提供优质的养老保险服务，致力于帮助客户在退休后过上安逸的生活。我们拥有一支经验丰富的团队，为客户提供全方位的保险咨询和服务。\n\n\\d+\\. 问：什么是养老保险？它的主要作用是什么？\n   答：养老保险是一种保险产品，旨在帮助个人在退休后获得稳定的收入来源，以应对生活中的各种支出，如医疗费用、日常开支等。它的主要作用是确保客户在老年时期有经济保障，过上安逸的生活。')

In [6]:
len(docs)

51

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [9]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
os.environ['OPENAI_API_KEY'] = 'sk-abcd'
os.environ['OPENAI_BASE_URL'] = 'https://api.xiaoai.plus/v1'
db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [29]:
query = "是否提供咨询服务？"

In [30]:
answer_list = db.similarity_search(query)

In [31]:
for ans in answer_list:
    print(ans.page_content + "\n")

问：安逸养老保险公司是否提供免费的保险咨询服务？
    答：是的，安逸养老保险公司提供免费的保险咨询服务。如果您对养老保险有任何疑问或需要专业的建议，您可以随时联系我们的销售顾问。他们将为您提供详细的解答和个性化的建议，帮助您了解养老保险的相关知识，并选择最适合您的保险产品。

\d+\. 问：安逸养老保险公司是否提供退保服务？
    答：是的，安逸养老保险公司提供退保服务。如果您在购买保险后需要退保，您可以联系我们的客户服务部门或销售顾问，提出退保申请。我们将根据保险合同的相关条款和规定，协助您办理退保手续。请注意，退保可能会涉及一定的手续费或退保价值的调整，请与我们的销售顾问咨询具体的退保政策和费用信息。

问：安逸养老保险公司是否有合作医院或医疗机构？
    答：是的，安逸养老保险公司与一些合作医院或医疗机构建立了合作关系。这些合作医院或医疗机构可以为我们的客户提供优先就诊、特殊待遇或其他相关服务。具体的合作医院或医疗机构名单可以在我们的官方网站或与销售顾问咨询时获取。我们致力于为客户提供全方位的保险保障和健康服务。

\d+\. 问：安逸养老保险公司是否提供养老规划咨询服务？
    答：是的，安逸养老保险公司提供养老规划咨询服务。我们的销售顾问可以根据您的年龄、收入、退休计划等因素，为您制定个性化的养老规划方案。养老规划涉及到财务规划、投资规划、退休收入规划等方面，我们将为您提供专业的建议和指导，帮助您实现理想的退休生活。

问：安逸养老保险公司是否提供保单贷款服务？
    答：是的，安逸养老保险公司提供保单贷款服务。如果您需要在保险合同有效期内获得一定的资金支持，您可以将保单作为抵押物向我们申请贷款。我们将根据您的保单价值和相关规定，评估贷款金额和利率。保单贷款可以帮助您应对紧急资金需求或其他个人经济需求。

\d+\. 问：安逸养老保险公司是否提供免费的保险理赔服务？
    答：是的，安逸养老保险公司提供免费的保险理赔服务。如果您需要申请保险理赔，您可以联系我们的客户服务部门或销售顾问，提供相关的索赔材料和证明。我们的理赔专员将会对您的申请进行审核，并尽快处理您的理赔请求。我们致力于为客户提供高效、公正和专业的保险理赔服务。

问：如果我需要索赔，应该如何操作？
    答：如果您需要索赔，您可以与我们的客户服务团队联系，提供相关的索赔申请和

In [32]:
db.save_local("real_insurance_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [50]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [51]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001AA87991B10>, search_kwargs={'k': 3})

In [52]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

问：安逸养老保险公司是否提供免费的保险咨询服务？
    答：是的，安逸养老保险公司提供免费的保险咨询服务。如果您对养老保险有任何疑问或需要专业的建议，您可以随时联系我们的销售顾问。他们将为您提供详细的解答和个性化的建议，帮助您了解养老保险的相关知识，并选择最适合您的保险产品。

\d+\. 问：安逸养老保险公司是否提供退保服务？
    答：是的，安逸养老保险公司提供退保服务。如果您在购买保险后需要退保，您可以联系我们的客户服务部门或销售顾问，提出退保申请。我们将根据保险合同的相关条款和规定，协助您办理退保手续。请注意，退保可能会涉及一定的手续费或退保价值的调整，请与我们的销售顾问咨询具体的退保政策和费用信息。

问：安逸养老保险公司是否有合作医院或医疗机构？
    答：是的，安逸养老保险公司与一些合作医院或医疗机构建立了合作关系。这些合作医院或医疗机构可以为我们的客户提供优先就诊、特殊待遇或其他相关服务。具体的合作医院或医疗机构名单可以在我们的官方网站或与销售顾问咨询时获取。我们致力于为客户提供全方位的保险保障和健康服务。

\d+\. 问：安逸养老保险公司是否提供养老规划咨询服务？
    答：是的，安逸养老保险公司提供养老规划咨询服务。我们的销售顾问可以根据您的年龄、收入、退休计划等因素，为您制定个性化的养老规划方案。养老规划涉及到财务规划、投资规划、退休收入规划等方面，我们将为您提供专业的建议和指导，帮助您实现理想的退休生活。

问：安逸养老保险公司是否提供保单贷款服务？
    答：是的，安逸养老保险公司提供保单贷款服务。如果您需要在保险合同有效期内获得一定的资金支持，您可以将保单作为抵押物向我们申请贷款。我们将根据您的保单价值和相关规定，评估贷款金额和利率。保单贷款可以帮助您应对紧急资金需求或其他个人经济需求。

\d+\. 问：安逸养老保险公司是否提供免费的保险理赔服务？
    答：是的，安逸养老保险公司提供免费的保险理赔服务。如果您需要申请保险理赔，您可以联系我们的客户服务部门或销售顾问，提供相关的索赔材料和证明。我们的理赔专员将会对您的申请进行审核，并尽快处理您的理赔请求。我们致力于为客户提供高效、公正和专业的保险理赔服务。



In [53]:
docs = topK_retriever.get_relevant_documents("受益怎么样")

In [54]:
for doc in docs:
    print(doc.page_content + "\n")

问：如何了解养老保险的最新信息和政策变动？
    答：我们会定期向客户发送最新的养老保险信息和政策变动通知。您也可以通过我们的官方网站、客户服务热线或移动应用程序获取最新的信息。此外，我们的销售顾问也会定期与您保持联系，向您提供最新的产品信息和政策解读。

\d+\. 问：养老保险是否可以终身受益？
    答：养老保险通常是终身受益的，意味着一旦购买并支付保费，您将享受终身的保险保障。这意味着在您退休后，您可以按照保险合同的约定获得养老金或一次性保险金。我们的销售顾问可以帮助您选择适合您需求的终身养老保险产品。

问：养老保险是否可以购买额外的税收优惠计划？
    答：一般情况下，养老保险本身并不提供额外的税收优惠计划。然而，根据您所在的地区和税法规定，您可能有资格享受养老保险相关的税收优惠。我们建议您咨询专业税务顾问或与当地税务机构联系，了解适用于您的个人情况的税收政策和优惠。

\d+\. 问：如果我在养老保险期间去世，我的受益人能否领取保险金？
    答：如果您在养老保险期间去世，根据保险合同的规定，您的受益人通常可以领取相应的保险金。保险金的领取方式和金额将根据您所购买的保险产品和合同条款来确定。我们的销售顾问可以为您详细解释受益人权益和保险金领取流程。

问：如果我在养老保险期间需要更改受益人，是否可以进行更改？
    答：一般情况下，您可以在养老保险期间更改受益人。您可以与我们的销售顾问联系，了解相关的受益人更改程序和要求。在进行受益人更改时，您可能需要填写相应的表格并提供必要的文件和证明。

\d+\. 问：养老保险是否可以购买额外的意外伤害保险？
    答：是的，养老保险可以与额外的意外伤害保险结合购买。意外伤害保险可以为您提供在退休后可能发生的意外事故中的经济保障和医疗费用补偿。结合养老保险和意外伤害保险可以全面保障您的经济和健康。我们的销售顾问可以为您提供个性化的保险规划建议。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [57]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.5}
)

In [58]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

问：安逸养老保险公司是否提供免费的保险咨询服务？
    答：是的，安逸养老保险公司提供免费的保险咨询服务。如果您对养老保险有任何疑问或需要专业的建议，您可以随时联系我们的销售顾问。他们将为您提供详细的解答和个性化的建议，帮助您了解养老保险的相关知识，并选择最适合您的保险产品。

\d+\. 问：安逸养老保险公司是否提供退保服务？
    答：是的，安逸养老保险公司提供退保服务。如果您在购买保险后需要退保，您可以联系我们的客户服务部门或销售顾问，提出退保申请。我们将根据保险合同的相关条款和规定，协助您办理退保手续。请注意，退保可能会涉及一定的手续费或退保价值的调整，请与我们的销售顾问咨询具体的退保政策和费用信息。

问：安逸养老保险公司是否有合作医院或医疗机构？
    答：是的，安逸养老保险公司与一些合作医院或医疗机构建立了合作关系。这些合作医院或医疗机构可以为我们的客户提供优先就诊、特殊待遇或其他相关服务。具体的合作医院或医疗机构名单可以在我们的官方网站或与销售顾问咨询时获取。我们致力于为客户提供全方位的保险保障和健康服务。

\d+\. 问：安逸养老保险公司是否提供养老规划咨询服务？
    答：是的，安逸养老保险公司提供养老规划咨询服务。我们的销售顾问可以根据您的年龄、收入、退休计划等因素，为您制定个性化的养老规划方案。养老规划涉及到财务规划、投资规划、退休收入规划等方面，我们将为您提供专业的建议和指导，帮助您实现理想的退休生活。

问：安逸养老保险公司是否提供保单贷款服务？
    答：是的，安逸养老保险公司提供保单贷款服务。如果您需要在保险合同有效期内获得一定的资金支持，您可以将保单作为抵押物向我们申请贷款。我们将根据您的保单价值和相关规定，评估贷款金额和利率。保单贷款可以帮助您应对紧急资金需求或其他个人经济需求。

\d+\. 问：安逸养老保险公司是否提供免费的保险理赔服务？
    答：是的，安逸养老保险公司提供免费的保险理赔服务。如果您需要申请保险理赔，您可以联系我们的客户服务部门或销售顾问，提供相关的索赔材料和证明。我们的理赔专员将会对您的申请进行审核，并尽快处理您的理赔请求。我们致力于为客户提供高效、公正和专业的保险理赔服务。

问：如果我需要索赔，应该如何操作？
    答：如果您需要索赔，您可以与我们的客户服务团队联系，提供相关的索赔申请和

### 提取向量数据库中的`销售回答`

In [59]:
docs = retriever.get_relevant_documents(query)

In [60]:
docs[0].page_content

'问：安逸养老保险公司是否提供免费的保险咨询服务？\n    答：是的，安逸养老保险公司提供免费的保险咨询服务。如果您对养老保险有任何疑问或需要专业的建议，您可以随时联系我们的销售顾问。他们将为您提供详细的解答和个性化的建议，帮助您了解养老保险的相关知识，并选择最适合您的保险产品。\n\n\\d+\\. 问：安逸养老保险公司是否提供退保服务？\n    答：是的，安逸养老保险公司提供退保服务。如果您在购买保险后需要退保，您可以联系我们的客户服务部门或销售顾问，提出退保申请。我们将根据保险合同的相关条款和规定，协助您办理退保手续。请注意，退保可能会涉及一定的手续费或退保价值的调整，请与我们的销售顾问咨询具体的退保政策和费用信息。'

In [61]:
docs[0].page_content.split("[销售回答] ")

['问：安逸养老保险公司是否提供免费的保险咨询服务？\n    答：是的，安逸养老保险公司提供免费的保险咨询服务。如果您对养老保险有任何疑问或需要专业的建议，您可以随时联系我们的销售顾问。他们将为您提供详细的解答和个性化的建议，帮助您了解养老保险的相关知识，并选择最适合您的保险产品。\n\n\\d+\\. 问：安逸养老保险公司是否提供退保服务？\n    答：是的，安逸养老保险公司提供退保服务。如果您在购买保险后需要退保，您可以联系我们的客户服务部门或销售顾问，提出退保申请。我们将根据保险合同的相关条款和规定，协助您办理退保手续。请注意，退保可能会涉及一定的手续费或退保价值的调整，请与我们的销售顾问咨询具体的退保政策和费用信息。']

In [62]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [63]:
ans

'问：安逸养老保险公司是否提供免费的保险咨询服务？\n    答：是的，安逸养老保险公司提供免费的保险咨询服务。如果您对养老保险有任何疑问或需要专业的建议，您可以随时联系我们的销售顾问。他们将为您提供详细的解答和个性化的建议，帮助您了解养老保险的相关知识，并选择最适合您的保险产品。\n\n\\d+\\. 问：安逸养老保险公司是否提供退保服务？\n    答：是的，安逸养老保险公司提供退保服务。如果您在购买保险后需要退保，您可以联系我们的客户服务部门或销售顾问，提出退保申请。我们将根据保险合同的相关条款和规定，协助您办理退保手续。请注意，退保可能会涉及一定的手续费或退保价值的调整，请与我们的销售顾问咨询具体的退保政策和费用信息。'

#### 尝试各种问题

In [67]:
from typing import List

def sales(query: str, score_threshold: float=0.5) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [68]:
query = "我想可以随时更换受益人的"

print(sales(query))

['问：如果我购买了养老保险后，需要更改受益人信息，是否可以办理修改？\n    答：如果您购买了养老保险后，需要更改受益人信息，一般情况下是可以办理修改的。具体的修改受益人信息的政策和程序将根据保险公司的规定而有所不同。一般来说，您可以与我们的销售顾问联系，提出修改受益人信息的要求，并根据保险公司的要求提供相应的文件和证明。我们的销售顾问可以帮助您了解具体的修改受益人信息的流程和要求。', '问：如果我在养老保险期间需要更改受益人，是否可以进行更改？\n    答：一般情况下，您可以在养老保险期间更改受益人。您可以与我们的销售顾问联系，了解相关的受益人更改程序和要求。在进行受益人更改时，您可能需要填写相应的表格并提供必要的文件和证明。\n\n\\d+\\. 问：养老保险是否可以购买额外的意外伤害保险？\n    答：是的，养老保险可以与额外的意外伤害保险结合购买。意外伤害保险可以为您提供在退休后可能发生的意外事故中的经济保障和医疗费用补偿。结合养老保险和意外伤害保险可以全面保障您的经济和健康。我们的销售顾问可以为您提供个性化的保险规划建议。', '问：购买养老保险时，是否可以选择不同的投保人和被保险人？\n    答：购买养老保险时，一般可以选择不同的投保人和被保险人。投保人是购买保险的人，负责支付保费和签署保险合同。被保险人是享受保险保障的人，通常是投保人本人或者是投保人指定的受益人。在某些情况下，投保人和被保险人可以是同一个人，也可以是不同的人。我们的销售顾问可以帮助您了解不同投保人和被保险人组合的选择，以满足您的个人需求和保险计划。', '问：养老保险是否可以购买多份保单？\n    答：是的，您可以购买多份养老保险保单。购买多份保单可以增加您的保险保障和退休收入。每份保单的保费和保险金额将根据您的选择而定。我们的销售顾问可以帮助您制定个性化的保险规划，以满足您的需求和目标。\n\n\\d+\\. 问：养老保险是否可以在保险期限内更改受益人？\n    答：一般情况下，养老保险是可以在保险期限内更改受益人的。如果您有需要更改受益人的情况，您可以与我们联系，提供相关的申请和身份证明文件。我们将根据保险合同的规定帮助您完成受益人的更改手续。']


In [69]:
print(sales(query, 0.7))

['问：如果我购买了养老保险后，需要更改受益人信息，是否可以办理修改？\n    答：如果您购买了养老保险后，需要更改受益人信息，一般情况下是可以办理修改的。具体的修改受益人信息的政策和程序将根据保险公司的规定而有所不同。一般来说，您可以与我们的销售顾问联系，提出修改受益人信息的要求，并根据保险公司的要求提供相应的文件和证明。我们的销售顾问可以帮助您了解具体的修改受益人信息的流程和要求。', '问：如果我在养老保险期间需要更改受益人，是否可以进行更改？\n    答：一般情况下，您可以在养老保险期间更改受益人。您可以与我们的销售顾问联系，了解相关的受益人更改程序和要求。在进行受益人更改时，您可能需要填写相应的表格并提供必要的文件和证明。\n\n\\d+\\. 问：养老保险是否可以购买额外的意外伤害保险？\n    答：是的，养老保险可以与额外的意外伤害保险结合购买。意外伤害保险可以为您提供在退休后可能发生的意外事故中的经济保障和医疗费用补偿。结合养老保险和意外伤害保险可以全面保障您的经济和健康。我们的销售顾问可以为您提供个性化的保险规划建议。', '问：购买养老保险时，是否可以选择不同的投保人和被保险人？\n    答：购买养老保险时，一般可以选择不同的投保人和被保险人。投保人是购买保险的人，负责支付保费和签署保险合同。被保险人是享受保险保障的人，通常是投保人本人或者是投保人指定的受益人。在某些情况下，投保人和被保险人可以是同一个人，也可以是不同的人。我们的销售顾问可以帮助您了解不同投保人和被保险人组合的选择，以满足您的个人需求和保险计划。']


In [72]:
query = "收益更多的"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.6 ans: {sales(query, 0.6)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: ['问：养老保险是否可以购买额外的退休金计划？\n    答：养老保险的主要目的是为了提供退休后的经济保障，一般不包含额外的退休金计划。然而，某些养老保险产品可能提供附加的退休金计划选项，以增加您的退休收入。我们的销售顾问可以为您介绍适合您需求的产品和退休金计划。\n\n\\d+\\. 问：养老保险是否可以购买额外的养老金递增选项？\n    答：一般情况下，养老保险的养老金是在购买时确定的，不可以随意递增。然而，某些养老保险产品可能提供养老金递增选项或养老金递增 rider，以应对通货膨胀等因素的影响。我们的销售顾问可以为您详细解释递增选项和其对您的养老金的影响。', '问：养老保险是否可以购买额外的税收优惠计划？\n    答：一般情况下，养老保险本身并不提供额外的税收优惠计划。然而，根据您所在的地区和税法规定，您可能有资格享受养老保险相关的税收优惠。我们建议您咨询专业税务顾问或与当地税务机构联系，了解适用于您的个人情况的税收政策和优惠。\n\n\\d+\\. 问：如果我在养老保险期间去世，我的受益人能否领取保险金？\n    答：如果您在养老保险期间去世，根据保险合同的规定，您的受益人通常可以领取相应的保险金。保险金的领取方式和金额将根据您所购买的保险产品和合同条款来确定。我们的销售顾问可以为您详细解释受益人权益和保险金领取流程。', '问：养老保险是否可以购买多份保单？\n    答：是的，您可以购买多份养老保险保单。购买多份保单可以增加您的保险保障和退休收入。每份保单的保费和保险金额将根据您的选择而定。我们的销售顾问可以帮助您制定个性化的保险规划，以满足您的需求和目标。\n\n\\d+\\. 问：养老保险是否可以在保险期限内更改受益人？\n    答：一般情况下，养老保险是可以在保险期限内更改受益人的。如果您有需要更改受益人的情况，您可以与我们联系，提供相关的申请和身份证明文件。我们将根据保险合同的规定帮助您完成受益人的更改手续。', '问：如何了解养老保险的最新信息和政策变动？\n    答：我们会定期向客户发送最新的养老保险信息和政策变动通知。您也可以通过我们的官方网站、客户服务热线或移动应用程序获取最新的信息。此外，我们的销售顾问也会定期与您保持联系，向您提供最新的产品信息和政策解读。\n\n\\d+\\. 问：养老保险

#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [75]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.6}))

In [76]:
qa_chain({"query": "和社保冲突吗？"})

{'query': '和社保冲突吗？',
 'result': '购买养老保险并不会和社保冲突。社会保险（社保）是国家强制性的保障制度，旨在为参保人提供基本的养老、医疗、失业等保障。而养老保险是商业保险的一种，属于个人自愿购买，旨在为个人提供额外的养老金储备和保障。两者可以同时存在，商业养老保险可以作为补充养老保障，为个人退休后的生活提供更多的财务支持和保障。因此，购买养老保险是在社保基础上的补充，有助于实现更全面的养老规划。'}

In [83]:
pip install -U langchain-openai


  Using cached openai-1.37.0-py3-none-any.whl.metadata (22 kB)
Using cached openai-1.37.0-py3-none-any.whl (337 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.30.1
    Uninstalling openai-1.30.1:
      Successfully uninstalled openai-1.30.1


## 加载 FAISS 向量数据库已有结果

In [81]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
db = FAISS.load_local("real_insurance_sale", OpenAIEmbeddings(),allow_dangerous_deserialization=True)

In [84]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [85]:
qa_chain({"query": "我想买市场上回报率最高的养老保险"})

C:\ProgramData\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买市场上回报率最高的养老保险',
 'result': '对不起，我无法提供具体的养老保险产品推荐，因为这需要考虑到各种因素，包括你的年龄、健康状况、收入等。建议你咨询专业的保险顾问或者进行深入的市场研究来找到最适合你的养老保险产品。'}

In [37]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [87]:
qa_chain({"query": "我想买市场上回报率最高的养老保险"})

C:\ProgramData\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买市场上回报率最高的养老保险',
 'result': '抱歉，作为一个AI，我无法提供最新的市场数据或推荐具体的保险产品。建议你咨询专业的保险顾问或进行深入的市场研究来找到最适合你的养老保险。'}

In [39]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [88]:
result = qa_chain({"query": "你们有市场上回报率最高的养老保险吗？"})

C:\ProgramData\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [89]:
result

{'query': '你们有市场上回报率最高的养老保险吗？',
 'result': '抱歉，我不能提供具体的保险产品信息，因为我是一个AI助手，无法实时更新市场上的保险产品。建议您咨询专业的保险顾问或者查阅相关保险公司的官方网站以获取最新的养老保险产品信息。'}